## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-20-18-47-42 +0000,startribune,Live: Farm Aid 40 kicks off at Huntington Bank...,https://www.startribune.com/live-farm-aid-40-k...
1,2025-09-20-18-41-37 +0000,nypost,White House sending two planeloads of staffers...,https://nypost.com/2025/09/20/us-news/white-ho...
2,2025-09-20-18-34-39 +0000,nyt,White House Outlines a TikTok Deal With a U.S....,https://www.nytimes.com/2025/09/20/technology/...
3,2025-09-20-18-33-09 +0000,latimes,Corrections officers at Norco prison make plea...,https://www.latimes.com/california/story/2025-...
4,2025-09-20-18-31-35 +0000,nyt,Can France’s Palestinian Proposal Change 75 Ye...,https://www.nytimes.com/2025/09/20/world/middl...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2445/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
60,trump,49
16,kirk,19
97,new,15
15,charlie,15
42,kimmel,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
169,2025-09-20-00-32-00 +0000,wsj,A top federal prosecutor who has been under pr...,https://www.wsj.com/politics/policy/trump-call...,143
161,2025-09-20-01-00-00 +0000,wsj,Charlie Kirk’s widow Erika Kirk will become th...,https://www.wsj.com/us-news/erika-kirk-charlie...,106
182,2025-09-19-23-29-42 +0000,nypost,Jimmy Kimmel foresaw his demise after second T...,https://nypost.com/2025/09/19/us-news/jimmy-ki...,105
55,2025-09-20-14-54-49 +0000,nypost,Trump says he axed US attorney investigating N...,https://nypost.com/2025/09/20/us-news/trump-sa...,105
173,2025-09-20-00-11-30 +0000,latimes,"Lopez: If he ever gets his job back, I have ju...",https://www.latimes.com/california/story/2025-...,98


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
169,143,2025-09-20-00-32-00 +0000,wsj,A top federal prosecutor who has been under pr...,https://www.wsj.com/politics/policy/trump-call...
161,91,2025-09-20-01-00-00 +0000,wsj,Charlie Kirk’s widow Erika Kirk will become th...,https://www.wsj.com/us-news/erika-kirk-charlie...
247,58,2025-09-19-19-37-40 +0000,nypost,"Ted Cruz rips FCC’s Brendan Carr, compares him...",https://nypost.com/2025/09/19/business/ted-cru...
212,42,2025-09-19-21-31-00 +0000,wsj,Senate Democrats blocked Republicans’ stopgap ...,https://www.wsj.com/politics/policy/government...
165,37,2025-09-20-00-35-15 +0000,nypost,"Dem mayor, anti-ICE protesters hit with tear g...",https://nypost.com/2025/09/19/us-news/anti-ice...
232,35,2025-09-19-20-21-00 +0000,wsj,Trump administration officials are in discussi...,https://www.wsj.com/politics/national-security...
120,32,2025-09-20-08-21-49 +0000,nypost,"Russia launches large-scale attack on Ukraine,...",https://nypost.com/2025/09/20/world-news/russi...
48,31,2025-09-20-15-22-28 +0000,nypost,Q&A: NYC Mayor Eric Adams calls Jews his secre...,https://nypost.com/2025/09/20/us-news/q-amp-a-...
51,30,2025-09-20-15-16-40 +0000,nypost,NJ man who lost dad on 9/11 thinks he was deni...,https://nypost.com/2025/09/20/us-news/nj-man-w...
181,30,2025-09-19-23-30-00 +0000,wsj,Health Secretary Robert F. Kennedy Jr.’s handp...,https://www.wsj.com/politics/policy/rfk-jr-cdc...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
